# Example full run

An example of a complete run of the lephare with all stages required to estimate redshift.

Define the two environment variables necessary for Le Phare (directory of LEPHARE package, directory in which to store the libraries) and define the directory in which you run the code. 

In [ ]:
import os
import lephare as lp
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

## Set up the parameters


Read the parameter file and store the keywords in a keywords map.

In [ ]:
config_file = "./data/COSMOS.para"
keymap = lp.read_config(config_file)
# Get the auxiliary files required.
lp.data_retrieval.get_auxiliary_data(
    keymap=keymap, additional_files=["examples/COSMOS.in", "examples/config.yml", "examples/output.para"]
)

Modification of three keywords of the parameter file. Verbose must be NO in the notebook.

In [ ]:
keymap["ZPHOTLIB"] = lp.keyword("ZPHOTLIB", "VISTA_COSMOS_FREE,ALLSTAR_COSMOS,QSO_COSMOS")
keymap["SPEC_OUT"] = lp.keyword("SPEC_OUT", "save_spec")
keymap["VERBOSE"] = lp.keyword("VERBOSE", "NO")
keymap["AUTO_ADAPT"] = lp.keyword("AUTO_ADAPT", "YES")

If adaption of the zero-points is turned off and you want to apply your own shift

In [ ]:
# keymap["AUTO_ADAPT"]= keyword("AUTO_ADAPT","NO")
# keymap["APPLY_SYSSHIFT"]= keyword("APPLY_SYSSHIFT","0.049,-0.013,-0.055,-0.065,-0.042,-0.044,-0.065,-0.0156,-0.002,0.052,-0.006,0.071,0.055,0.036,0.036,0.054,0.088,0.019,-0.154,0.040,0.044,0.060,0.045,0.022,0.062,0.033,0.015,0.012,0.0,0.0]

## Create filter library

Read the filter names to be used in COSMOS.para and generate the filter file

First, you can use the standard method with the list of filters in the parameter file. The filters are store in the LEPHAREDIR/filt directory. You can pass either the config file or the keymap as argument

In [ ]:
filterLib = lp.Filter(config_file=config_file)
# uncomment to test passing the keymap
# filterLib = Filter(config_keymap=keymap)
filterLib.run()

Second, you can use the filterSvc helper class to gain more freedom. This class allows to retrieve the list of filt objetcs, independently of writing them on file.

In [ ]:
filterLib = lp.FilterSvc.from_config(config_file)
filter_output = os.path.join(os.environ["LEPHAREWORK"], "filt", keymap["FILTER_FILE"].value)
lp.write_output_filter(filter_output + ".dat", filter_output + ".doc", filterLib)

It also allows to load the filters from a yaml file, with the possibility to query the SVO service for filters

In [ ]:
!ls $LEPHAREDIR/examples/

In [ ]:
filterLibSVO = lp.FilterSvc.from_yaml(f"{lp.LEPHAREDIR}/examples/config.yml")
filter_output = os.path.join(os.environ["LEPHAREWORK"], "filt", keymap["FILTER_FILE"].value)
lp.write_output_filter(filter_output + "_svo.dat", filter_output + "_svo.doc", filterLib)

Plot the filters

In [ ]:
fig = plt.figure(figsize=(15, 8))
for f, fsvo in zip(filterLib, filterLibSVO):
    d = f.data()
    plt.semilogx(d[0], d[1] / d[1].max())
    dsvo = fsvo.data()
    plt.semilogx(dsvo[0], dsvo[1] / dsvo[1].max(), ".")

In [ ]:
# filter_output = os.path.join(os.environ["LEPHAREWORK"],"filt", filterLib.keymap['FILTER_FILE'].value + ".dat")
filters = np.loadtxt(
    filter_output + ".dat", dtype={"names": ("lamb", "val", "bid"), "formats": (float, float, int)}
)
plt.loglog(filters["lamb"], filters["val"])
plt.xlabel("wavelength");

## Create SED library

SED objects represent SED templates belonging to one of the three possible classes "STAR", "QSO" (for AGN type of objects), and "GAL" for galaxies. SED templates available with LePhare can be found under the `sed` directory. 

In [ ]:
sedlib = lp.Sedtolib(config_keymap=keymap)

In [ ]:
sedlib.run(typ="STAR", star_sed="$LEPHAREDIR/sed/STAR/STAR_MOD_ALL.list")

In [ ]:
sedlib.run(typ="QSO", qso_sed="$LEPHAREDIR/sed/QSO/SALVATO09/AGN_MOD.list", gal_lib="LIB_QSO")

In [ ]:
sedlib.run(typ="GAL", gal_sed="$LEPHAREDIR/examples/COSMOS_MOD.list", gal_lib="LIB_GAL")

## Create a magnitude library

Use the SED library to create a magnitude library

In [ ]:
maglib = lp.MagGal(config_keymap=keymap)

In [ ]:
maglib.run(typ="STAR", lib_ascii="YES", star_lib_out="ALLSTAR_COSMOS")

In [ ]:
maglib.run(
    typ="QSO",
    lib_ascii="YES",
    mod_extinc="0,1000",
    eb_v="0.,0.1,0.2,0.3",
    extinc_law="SB_calzetti.dat",
    qso_lib_in="LIB_QSO",
)

In [ ]:
maglib.run(
    typ="GAL",
    lib_ascii="YES",
    gal_lib_in="LIB_GAL",
    gal_lib_out="VISTA_COSMOS_FREE",
    mod_extinc="18,26,26,33,26,33,26,33",
    extinc_law="SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat",
    em_lines="EMP_UV",
    em_dispersion="0.5,0.75,1.,1.5,2.",
)

## Run the photoz

Read the parameter file and store the keywords. Example with the modification of three keywords of the parameter file. Verbose must be NO in the notebook.

In [ ]:
keymap = lp.read_config(config_file)
keymap["ZPHOTLIB"] = lp.keyword("ZPHOTLIB", "VISTA_COSMOS_FREE,ALLSTAR_COSMOS,QSO_COSMOS")
keymap["SPEC_OUT"] = lp.keyword("SPEC_OUT", "save_spec")
keymap["VERBOSE"] = lp.keyword("VERBOSE", "NO")

Instantiate the photo-z run with the above parameters.

In [ ]:
photz = lp.PhotoZ(keymap)

Read the input file with the following information: id, flux and associated uncertainties in all bands, a context indicating which bands to use in the fit (0 indicates all bands), and a spectrocopic redshift if it exists. 

In [ ]:
cat = np.loadtxt(f"{lp.LEPHAREDIR}/examples/COSMOS.in")
id = cat[:, 0]
fluxes = cat[:, 1:60:2]
efluxes = cat[:, 2:61:2]
context = cat[:, 61]
zspec = cat[:, 62]
print("Check format with context and zspec :", context, zspec)

Create a list of sources with a spec-z. Use for the zero-point training or any validation run.


In [ ]:
srclist = []
zspec_mask = np.logical_and(zspec > 0.01, zspec < 6)
for i in np.where(zspec_mask)[0]:
    oneObj = lp.onesource(i, photz.gridz)
    oneObj.readsource(str(id[i]), fluxes[i, :], efluxes[i, :], int(context[i]), zspec[i], " ")
    photz.prep_data(oneObj)
    srclist.append(oneObj)
print("Sources with a spec-z: ", len(srclist))

Derive the zero-points offsets. Here, this option is commented and zero-points are assumed to be know.


In [ ]:
a0, a1 = photz.run_autoadapt(srclist)
offsets = ",".join(np.array(a0).astype(str))
offsets = "# Offsets from auto-adapt: " + offsets + "\n"
print(offsets)

Create the list of sources for which we want a photo-z. here, limited to the 100 first sources.

In [ ]:
photozlist = []
for i in range(10):
    oneObj = lp.onesource(i, photz.gridz)
    oneObj.readsource(str(id[i]), fluxes[i, :], efluxes[i, :], int(context[i]), zspec[i], " ")
    photz.prep_data(oneObj)
    photozlist.append(oneObj)
print("Number of sources to be analysed: ", len(srclist))

Create the list of sources for which we want a photo-z. here, limited to the 100 first sources.

In [ ]:
photz.run_photoz(photozlist[:10], a0, a1)
# If adaption of the zero-points is turned off
# photz.run_photoz(photozlist[:100], [],[] )

Create output in fits


In [ ]:
t = photz.build_output_tables(photozlist[:10], para_out=None, filename="outputpython.fits")

In [ ]:
t[:5]

In [ ]:
t[:5]

Create one ascii file per object to produce the best-fit template plot

In [ ]:
import time

photz.write_outputs(photozlist[:10], int(time.time()))

Make the plot with all the file listed in save_spec

In [ ]:
from os import listdir
from os.path import isfile, join

listname = [f for f in listdir("save_spec") if isfile(join("save_spec", f))]
for namefile in listname:
    lp.plotspec("save_spec/" + str(namefile))